In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as ny

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID", axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
indep=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
dep=dataset['Purchased']

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep, dep, test_size=1/3, random_state=0)

In [13]:
indep.shape

(400, 3)

In [14]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [16]:
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

param_grid={'kernel':['rbf', 'linear', 'poly', 'sigmoid'], 'C':[10,100,1000,2000,3000], 'gamma':['auto','scale']}

grid=GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring='f1_weighted')
#to tune a model
grid.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [17]:
#to know the best tune parameter
re = grid.cv_results_
#print(re)

grid_prediction=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)
print(cm)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134

[[80  5]
 [ 7 42]]


In [18]:
#to know the f1 score
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_prediction, average='weighted')
print("The f1_macro value of best parameter is {}:".format(grid.best_params_),f1_macro)

The f1_macro value of best parameter is {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [19]:
#as a addition to know a receiver operator characteristic area under curve value to identify best classification
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,-1])

0.9539015606242497

In [20]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027343,0.003621,0.010831,0.002968,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
1,0.028499,0.003505,0.008210,0.001672,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
2,0.031196,0.003920,0.007818,0.002257,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
3,0.018599,0.002456,0.008264,0.001847,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,39
4,0.023855,0.002727,0.008628,0.001527,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
5,0.026689,0.004589,0.007359,0.001411,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
6,0.022646,0.003010,0.007383,0.002059,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
7,0.014958,0.001988,0.007292,0.000816,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.738916,0.753180,0.778067,0.766556,0.759879,0.013172,31
8,0.025691,0.002457,0.007565,0.001607,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,1
9,0.088363,0.003744,0.008286,0.001058,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21


In [21]:
age_input=float(input('Age'))
EstimatedSalary_input=float(input('EstimatedSalary'))
Gender_Male_input=float(input('Gender_Male'))

Age 26
EstimatedSalary 80000
Gender_Male 1


In [22]:
grid.predict([[age_input, EstimatedSalary_input, Gender_Male_input]])

array([1], dtype=int64)